# Tugas Besar 1 IF3270 Pembelajaran Mesin <br /> Feedforward Neural Network

## Kelompok 39

- Dzaky Satrio Nugroho - 13522059
- Julian Caleb Simandjuntak - 13522099
- Rafiki Prawhira Harianto - 13522065

In [1]:
# Import dulu
import numpy as np


In [7]:
# Fungsi Aktivasi 

# Fungsi linear
def linear(x: np.ndarray) -> np.ndarray:
    return x

# Fungsi ReLU
def relu(x: np.ndarray) -> np.ndarray:
    return np.maximum(0, x)

# Fungsi Sigmoid
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

# Fungsi Hyperbolic Tangent
def tanh(x: np.ndarray) -> np.ndarray:
    return np.tanh(x)

# Fungsi Softmax
def softmax(x: np.ndarray) -> np.ndarray:
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


In [6]:
# Fungsi Loss

# Mean Squared Error
def mse(y_pred: np.ndarray, y_true: np.ndarray) -> float:
    mse = np.sum((y_pred - y_true) ** 2) / len(y_true)
    return mse

# Binary Cross-Entropy
def bce(y_pred: np.ndarray, y_true: np.ndarray) -> float:
    bce = -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).mean()
    return bce

# Categorical Cross-Entropy
def cce(y_pred: np.ndarray, y_true: np.ndarray) -> float:
    cce = -1 / len(y_true) * np.sum(np.sum(y_true * np.log(y_pred)))
    return cce